In [1]:
from collections import namedtuple
import numpy as np
import math
import random
import pandas as pd
import os
import _json


In [49]:
Graph = namedtuple("Graph", ["nodes", "edges"])

nodes = ["A","B","C","D"]
edges = [
    ("A","B"),
    ("A","B"),
    ("A","C"),
    ("A","C"),
    ("A","D"),
    ("B","D"),
    ("C","D")
]

G = Graph(nodes,edges) 

adjacency_dict = {node:[] for node in G.nodes}

for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adjacency_dict[node1].append(node2)
    adjacency_dict[node2].append(node1)

adjacency_dict

ERROR! Session/line number was not unique in database. History logging moved to new session 1418


{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [50]:
[[5 for i in range(3)] for j in range(5)]

[[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5]]

In [51]:
nodes = range(4)
edges = [
    (0,1),
    (0,1),
    (0,2),
    (0,2),
    (0,3),
    (1,3),
    (2,3)
]

G = Graph(nodes,edges)

adj = [[0 for node in G.nodes] for node in G.nodes] 

for edge in G.edges:
    node1,node2 = edge[0], edge[1]
    adj[node1][node2] += 1
    adj[node2][node1] += 1
adj

[[0, 2, 2, 1], [2, 0, 0, 1], [2, 0, 0, 1], [1, 1, 1, 0]]

<h1>Generating an adjacency matrix for a graph</h1>



<p>We define the graph nodes and edges as follows:</p>
<img src="graphs.png" height="400" >
<p> Then we generate a graph object using namedtuple from the collections module. </p>
<p>We then define the edges manually to then generate an adjacency matrix</p>

<h3>Define the graph</h3>
<p>This is for version 1 of the experiment, where we used all 7 nodes, but after looking at preliminary data, we will proceed without the 7th/ centre node</p>

In [52]:
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,1),
    (0,5),
    (0,6),
    (1,0),
    (1,2),
    (1,6),
    (2,1),
    (2,3),
    (2,6),
    (3,2),
    (3,4),
    (3,6),
    (4,3),
    (4,5),
    (4,6),
    (5,4),
    (5,6),
    (5,0),
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [53]:
numberofNodes = len(G.nodes)
matrix_shape = (numberofNodes, numberofNodes)
adj_matrix = np.zeros(matrix_shape)
for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adj_matrix[node1][node2] +=1
    adj_matrix[node2][node1] +=1

adj_matrix = adj_matrix/2 #just to make everything 1s and 0s



In [54]:
adj_matrix

array([[0., 1., 0., 0., 0., 1., 1.],
       [1., 0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 1., 0., 1., 1.],
       [1., 0., 0., 0., 1., 0., 1.],
       [1., 1., 1., 1., 1., 1., 0.]])

<h1>Creating transition matrices</h1>

      
<p>The following adjacency matrix was generated by the above code for the graph(see below):</p>
<img src="graphs.png" height="400" >
      
         0   1   2   3   4   5   6
      0 [0., 1., 0., 0., 0., 1., 1.]
      1 [1., 0., 1., 0., 0., 0., 1.]
      2 [0., 1., 0., 1., 0., 0., 1.]
      3 [0., 0., 1., 0., 1., 0., 1.]
      4 [0., 0., 0., 1., 0., 1., 1.]
      5 [1., 0., 0., 0., 1., 0., 1.]
      6 [1., 1., 1., 1., 1., 1., 0.]

<p>Using this adjacency matrix, we can easily create the transition matrix based off the probabilities defined in the figure.</p>
<p>The stay probability of a node is defined by the looping arrow.</p>
   <p>This stay probability is simply the edge connecting the node to itself. This is represented by the <strong>diagonal</strong> of the adjacency matrix.</p>
<p>The transition probability of a node (to the next node) is simply then : (1 - stay probability of that node) / number of adjacent nodes.</p>

In [55]:
def generate_transitionMatrix(graph,stay_probability=0):
    """Function to create the transition matrix. We assume that the graph has already been created and the 
    relevant nodes and edges are clearly defined. """

    #get the number of nodes
    numberofNodes = len(graph.nodes)
    #get shape of the transition matrix 
    matrix_shape = (numberofNodes, numberofNodes)
    #create the adjacent matrix with zeros
    adj_matrix = np.zeros(matrix_shape)

    for edge in graph.edges:
        node1,node2 = edge[0],edge[1]
        adj_matrix[node1][node2] += 1
        adj_matrix[node2][node1] += 1
    adj_matrix = adj_matrix/2 #just to make everything 1s and 0s
    
    #lets keep the calculation of the transition probabilites dynamic based off the number of adjacent nodes from
    #a starting node and the stay probability of that starting node
    #therefore what we should have is: transition probability = (1 - stay probability) / number of adjacent nodes

    np.fill_diagonal(adj_matrix,stay_probability) #set the stay probability in the adj matrix for the for loop 
    transition_matrix = np.copy(adj_matrix)

    #lets get the number of neighbours for each node by reading off the total number of 1s (adjacent nodes)
    for index,row in enumerate(adj_matrix):
        #we assume that the loops of a node to itself are already defined in the edges of the graph
        number_adjacent_nodes = np.count_nonzero(row == 1)
        transition_probability = (1 - stay_probability) / (number_adjacent_nodes)

        #adjust the decimal places
        transition_probability

        #replace the 1s (adjacent nodes) with their transition probability
        transition_matrix[index] = np.where(row ==1, f"{transition_probability:.2f}", row) #formatting the transition probability to two decimal places

    #set the stay probability of the matrix for each node 
    np.fill_diagonal(transition_matrix,stay_probability)
    #replace the diagonals with zeros for the adjacent matrix
    np.fill_diagonal(adj_matrix, 0)

    return adj_matrix, transition_matrix
        



In [56]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [57]:
colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=0.1)
colour_adjmatrix, colour_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 , 0.3 ],
        [0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 ],
        [0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.3 ],
        [0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.3 ],
        [0.3 , 0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 ],
        [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.1 ]]))

In [58]:
texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G, stay_probability=0.3)
texture_adjmatrix, texture_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.3 , 0.23, 0.  , 0.  , 0.  , 0.23, 0.23],
        [0.23, 0.3 , 0.23, 0.  , 0.  , 0.  , 0.23],
        [0.  , 0.23, 0.3 , 0.23, 0.  , 0.  , 0.23],
        [0.  , 0.  , 0.23, 0.3 , 0.23, 0.  , 0.23],
        [0.  , 0.  , 0.  , 0.23, 0.3 , 0.23, 0.23],
        [0.23, 0.  , 0.  , 0.  , 0.23, 0.3 , 0.23],
        [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.3 ]]))

In [59]:
shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G, stay_probability=0.5)
shape_adjmatrix, shape_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.5 , 0.17, 0.  , 0.  , 0.  , 0.17, 0.17],
        [0.17, 0.5 , 0.17, 0.  , 0.  , 0.  , 0.17],
        [0.  , 0.17, 0.5 , 0.17, 0.  , 0.  , 0.17],
        [0.  , 0.  , 0.17, 0.5 , 0.17, 0.  , 0.17],
        [0.  , 0.  , 0.  , 0.17, 0.5 , 0.17, 0.17],
        [0.17, 0.  , 0.  , 0.  , 0.17, 0.5 , 0.17],
        [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.5 ]]))

<h1>Creating Design Files - Pseudo Code for version 1</h1>
<p>Create an empty dataframe which should hold columns for</p>
    <ul>
    <li>Participant ID</li>
    <li>Block Number</li>
    <li>Trial Number</li>
    <li>Colour_stim</li>
    <li>Texture_stim</li>
    <li>Shape_stim</li>
    <li>Colour_choice</li>
    <li>Texture_choice</li>
    <li>Shape_stim</li>
    </ul>
<p>Set a variable for number of trials</p>
<p>Set a variable for number of participants for which we need to create design files</p>
<p>Set a flag for every 20th trial to trigger a choice trial.</p>
<p>Set up an array containing the three stay probabilities [0.1, 0.3, 0.5]. The stay probabilities need to be included as a parameter into the function to generate transition probabilities </p>
<p>Loop through the range of participants</p>
    <p>Pick a transition probability randomly for each of the dimensions. We do this to ensure participants  experience fast,medium and slow transitions of the dimensions </p>
    <p>Set up adjacency and transition matrices for each dimension using the functions with the chosen transition probs in the step above.</p>
    <p>Set up variables to hold current rows for colour, texture and shape</p>
    <p>Set up a variable for block, update this every 180 trials by +1. </p>
    <p>Loop through number of trials : </p>
        <ul>
        <li>Just for the first iteration, generate 3 random whole numbers between 1-7. These 3 numbers will be the index of the row of the transition matrix of each dimension. Record these 3 numbers as the current row</li>
        <li>These resulting numbers will be used to generate the first object (colour + texture + shape) </li>
        <li>From the second iteration onwards (for all the following shapes), generate 3 random numbers between 0 - 1.</li>
            <li>If the randomly generated number for all or any dimension, is greater than the stay probability of that dimension, we transition to the adjacent node. To transition, we do the following:</li>
            <li>To determine which node (so basically decide the next row) to transition to based off the current row, we look for adjacent nodes (columns) in the current row of the matrix where != 0.</li>
            <li>Note that sometimes the stay probabilities are greater than the transition probabilities.</li>
            <li> If the current row != 6 (7th row), then we follow the steps below to find the node to transition-</li>
            <li>If difference <= 0.4, transition to first neighbour, get column index, this number/index becomes current row. Update the respective dimension column(s) for that trial with the row number</li>
            <li>If difference >0.4 & <= 0.7, transition to second neighbour, get column index, this number/index becomes current row.</li>
            <li>If difference >0.7 , transition to third neighbour, get column index, this number/index becomes current row.</li>
            <li>If row == 6, then we randomly pick a neighbour to transition to.</li>
            <li>Every 20th trial, or rather If the trial%20 == 0, then we generate 3 random whole numbers for the choice columns for the choice trials  </li>
            <li>If the randomly generated number for all or any dimension, is less than stay probability, we don't transition. Therefore current row remains the same</li>
            <li>To include our choice trials, if trial % 20, we draw also draw 3 random numbers and add it to the colour_choice, texture_choice and shape_choice rows of the dataframe. Otherwise they remain NaNs</li>
            </ul>

In [60]:
import pandas as pd
import numpy as np
import random

In [ ]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [ ]:
def generateChoiceTrials(total_trials, n_choice_trials, start_trial, jitter_range):
    """Function generates an array containing choice trials for the participants. The start trial parameter is used to
    to define the first choice trial. The total trials are of the experiment. The parameter
    n_choicetrials is the number of choice trials the participant gets. We define the starting trial and a jitter range.
    The minimum interval between trials are calculated inside the function and a random jitter from 0 to 5 is added
    to the interval. This makes the trials less predictable"""

    #generate the random choice trials between 1 and 240 trials with the first choice trial appearing at trial 10 (start trial) or later
    #lets start with generating random trial numbers which are greater than 10 but less than 20
    #we start with a first trial being a number between 10 and 15
    # subsequent trials are generated if trial is greater than the previous trial and if the trial is between 10 and 20 increments from the previous trial
    #these trials must be a total of 18 trials and should be spaced out so they fill the span of 240 total trials

    minimum_interval = total_trials // n_choice_trials
    choice_trials = []


    for i in range(n_choice_trials):
        base = start_trial + i * minimum_interval
        jitter = random.randint(0, jitter_range)
        subsequent_trial = min(base + jitter, total_trials - 1)
        choice_trials.append(subsequent_trial)

    return choice_trials

a = generateChoiceTrials(200*8,18*8,10,5)    
a

[12,
 24,
 35,
 48,
 56,
 66,
 77,
 91,
 98,
 113,
 123,
 136,
 144,
 157,
 169,
 177,
 191,
 201,
 213,
 222,
 230,
 241,
 257,
 264,
 278,
 288,
 297,
 309,
 321,
 332,
 342,
 356,
 364,
 377,
 384,
 395,
 408,
 421,
 432,
 444,
 453,
 464,
 476,
 485,
 497,
 505,
 518,
 527,
 538,
 550,
 560,
 576,
 587,
 593,
 605,
 620,
 631,
 638,
 650,
 659,
 673,
 681,
 694,
 704,
 719,
 727,
 739,
 751,
 760,
 772,
 785,
 791,
 804,
 816,
 827,
 836,
 846,
 862,
 870,
 884,
 895,
 902,
 912,
 927,
 937,
 947,
 961,
 972,
 982,
 993,
 1005,
 1014,
 1023,
 1035,
 1047,
 1059,
 1071,
 1077,
 1091,
 1103,
 1110,
 1123,
 1133,
 1147,
 1156,
 1166,
 1177,
 1192,
 1198,
 1213,
 1221,
 1236,
 1245,
 1257,
 1267,
 1278,
 1288,
 1297,
 1310,
 1323,
 1330,
 1346,
 1356,
 1366,
 1374,
 1390,
 1399,
 1407,
 1420,
 1432,
 1443,
 1455,
 1462,
 1474,
 1488,
 1495,
 1506,
 1517,
 1533,
 1544,
 1553,
 1562,
 1572,
 1586]

Loop through all trials
find potential indices from trial 10 onwards where the distractor were not consistent across all dimensions within the 200 trials
record these indices
record the distractors as well for that indice
from the record of these indices, choose 18 of them each with a gap of 8-12 trials apart
repeat for other blocks




In [ ]:
master_df = []

nTrials = 200
nParticipants = 50
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block
# this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
possibleNeighbours = {
    0 : [0,1,5,6],
    1 : [1,0,2,6],
    2 : [2,1,3,6],
    3 : [3,2,4,6],
    4 : [4,3,5,6],
    5 : [5,4,0,6],
    6 : [0,1,2,3,4,5,6]
}




for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the stimulus
    "Distractor_shares_texture",
    "Distractor_shares_shape",
    "Distractor_shares_slow", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the stimulus, however just mapped based off the speed 
    "Distractor_shares_medium",
    "Distractor_shares_fast"
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                
                rowIndex += 1 

            rowIndex_start_of_block = rowIndex - nTrials
            eligible_trials = []
            nodes_preced = None

            for trial in range(10, nTrials):  # start at trial 10
                rowIndex_current = rowIndex_start_of_block + trial
                
                c_prev = df.loc[rowIndex_current-1, "Colour_stim"]
                t_prev = df.loc[rowIndex_current-1, "Texture_stim"]
                s_prev = df.loc[rowIndex_current-1, "Shape_stim"]
                nodes_preced = [c_prev,t_prev,s_prev]

                distractor_colour = random.randint(0,6)
                distractor_texture = random.randint(0,6)
                distractor_shape = random.randint(0,6)
                distractorNodeVals = [distractor_colour,distractor_texture,distractor_shape]

                allMatch = (
                    distractor_colour in possibleNeighbours[c_prev],
                    distractor_texture in possibleNeighbours[t_prev],
                    distractor_shape in possibleNeighbours[s_prev]
                )

                if sum(allMatch) < 3:  # eligible trial
                    eligible_trials.append((trial, distractor_colour, distractor_texture, distractor_shape))
            
            #print(eligible_trials)

            choice_trials = []
            last_choice = -999
            for trial, dcol, dtex, dsha in eligible_trials:
                if trial - last_choice >= random.randint(8, 12):
                    choice_trials.append((trial, dcol, dtex, dsha))
                    last_choice = trial
                    if len(choice_trials) == 18:
                        break
            #print(len(choice_trials))


            for idx, (trial, dcol, dtex, dsha) in enumerate(choice_trials, start=1):
                rowIndex_current = rowIndex_start_of_block + trial
                df.loc[rowIndex_current, "Colour_distractor"] = dcol
                df.loc[rowIndex_current, "Texture_distractor"] = dtex
                df.loc[rowIndex_current, "Shape_distractor"] = dsha
                df.loc[rowIndex_current, "Choice Trial Index"] = idx

                stimNodeVals = [
                df.loc[rowIndex_current, "Colour_stim"],
                df.loc[rowIndex_current, "Texture_stim"],
                df.loc[rowIndex_current, "Shape_stim"]
                ]
                colsToModify = ["Distractor_shares_colour", "Distractor_shares_texture","Distractor_shares_shape"]
                distractorNodeVals = [dcol, dtex, dsha]

                for j, val in enumerate(distractorNodeVals):
                    df.loc[rowIndex_current, colsToModify[j]] = 1 if val == stimNodeVals[j] else 0


                stimNodeDimensionSpeedMap = {
                    "Colour": stimNodeVals[0],
                    "Texture": stimNodeVals[1],
                    "Shape": stimNodeVals[2]
                }
                distractorNodeDimensionsMap = {
                    "Colour": distractorNodeVals[0],
                    "Texture": distractorNodeVals[1],
                    "Shape": distractorNodeVals[2]
                }
                df.loc[rowIndex_current, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                df.loc[rowIndex_current, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                df.loc[rowIndex_current, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]

                stimNodeSpeedVals = [
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]],
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]],
                    stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                ]
                distractorNodeSpeedVals = [
                    distractorNodeDimensionsMap[dimensionSpeeds["Slow"]],
                    distractorNodeDimensionsMap[dimensionSpeeds["Medium"]],
                    distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]
                ]
                speedColsToModify = ["Distractor_shares_slow", "Distractor_shares_medium", "Distractor_shares_fast"]

                for j, val in enumerate(distractorNodeSpeedVals):
                    df.loc[rowIndex_current, speedColsToModify[j]] = 1 if val == stimNodeSpeedVals[j] else 0

                consistentCols = ["Distractor_consistent_colour", "Distractor_consistent_texture", "Distractor_consistent_shape"]
                consistentSpeedCols = ["Distractor_consistent_slow", "Distractor_consistent_medium", "Distractor_consistent_fast"]

                nodes_preced = [
                    df.loc[rowIndex_current-1, "Colour_stim"],
                    df.loc[rowIndex_current-1, "Texture_stim"],
                    df.loc[rowIndex_current-1, "Shape_stim"]
                ]
                nodes_speed_preced = [
                    df.loc[rowIndex_current-1, "Slow_stim"],
                    df.loc[rowIndex_current-1, "Medium_stim"],
                    df.loc[rowIndex_current-1, "Fast_stim"]
                ]
                distractorSpeedVals = distractorNodeSpeedVals

                for j, val in enumerate(nodes_preced):
                    df.loc[rowIndex_current, consistentCols[j]] = 1 if distractorNodeVals[j] in possibleNeighbours[val] else 0

                for j, val in enumerate(nodes_speed_preced):
                    df.loc[rowIndex_current, consistentSpeedCols[j]] = 1 if distractorSpeedVals[j] in possibleNeighbours[val] else 0
                            

                    
        df.iloc[:,0] = participant + 1
        df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        #df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
                

                    

KeyboardInterrupt: 

<h3>Copy of above script - adjusted to ensure that target is an entire transition from previous object. Distractor is also an entire transition from previous object and is max only consistent across 2 dimensions or less. </h3>

<p>Ensure also that last trial of every block is a choice trial</p>

In [ ]:
master_df = []

nTrials = 200
nParticipants = 50
nBlocks = 8
nSessions = 2
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block
# this map is to check if the randomly generated nodes for a distractor in a choice trial are consistent with
    # the preceeding stimulus object in the graph structure. If the preceeding stimulus object has the nodes 1 1 1 (Colour,Texture,Shape)
    # and the distractor has the nodes 2 2 4 (Colour, Texture, Shape), then its nodes are consistent with the graph structure
    # as these nodes are neighbours. Therefore we populate the distractor_consistent_colour, distractor_consistent_texture with 1s and 0 for distractor_consistent_shape
possibleNeighbours = {
    0 : [0,1,5,6],
    1 : [1,0,2,6],
    2 : [2,1,3,6],
    3 : [3,2,4,6],
    4 : [4,3,5,6],
    5 : [5,4,0,6],
    6 : [0,1,2,3,4,5,6]
}




for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Session Number",
    "Trial Number",
    "Stay_Probability(Colour)",
    "Stay_Probability(Texture)",
    "Stay_Probability(Shape)",
    "Slow_stim", #shows the node of the slow dimension (e.g.: if colour is the slower changing dimension, the node will be shown here. This is the same as the value in "Colour_stim")
    "Medium_stim",
    "Fast_stim",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Choice Trial Index",
    "Colour_distractor",
    "Texture_distractor",
    "Shape_distractor",
    "Slow_distractor", #shows the node of the distractor in terms of the slow dimension (e.g.: if colour is the slower changing dimension for the stimulus, the node for colour of the distractor will be shown here. This is the same as the value in "Colour_distractor")
    "Medium_distractor",
    "Fast_distractor",
    "Target_shares_colour",
    "Target_shares_texture",
    "Target_shares_shape",
    "Target_shares_slow",
    "Target_shares_medium",
    "Target_shares_fast",
    "Distractor_consistent_colour",
    "Distractor_consistent_texture",
    "Distractor_consistent_shape",
    "Distractor_consistent_slow",
    "Distractor_consistent_medium",
    "Distractor_consistent_fast",
    "Distractor_shares_colour_target", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the target
    "Distractor_shares_texture_target",
    "Distractor_shares_shape_target",
    "Distractor_shares_slow_target", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the target, however just mapped based off the speed 
    "Distractor_shares_medium_target",
    "Distractor_shares_fast_target",
    "Distractor_shares_colour_object", #column with 1s and 0s to show if the randomly generated distractor has the same dimensions as the target
    "Distractor_shares_texture_object",
    "Distractor_shares_shape_object",
    "Distractor_shares_slow_object", #column with 1s and 0s to show if the randomly generated distractor shows the same dimensions as the target, however just mapped based off the speed 
    "Distractor_shares_medium_object",
    "Distractor_shares_fast_object",
    
    ]


    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)
    #assign the respective dimensions to the speeds in which they transition
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) ==0.3)[0][0]
    fast = np.argmin(stayProbs)

    # the speed map is to just map the nodes on to the dimension, based off the speed in which it changes
    # e.g. if colour has a stay prob of 0.1, this dimension changes faster, so a node of 2 in the Colour_stim column
    # would be mapped to the Stim_fast column
    dimensionSpeedMap = {
        0 : "Colour",
        1 : "Texture",
        2 : "Shape"
    }

    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow] ,
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    


    for session in range(nSessions):
        df = pd.DataFrame(columns=columns)
        rowIndex = 0
        for block in range(nBlocks):            
            currentRow_colour = None
            currentRow_shape = None
            currentRow_texture = None
            choiceTrialIndex = 1
            
            for trial in range(nTrials):
                df.loc[rowIndex,"Block Number"] = block + 1
                df.loc[rowIndex, "Session Number"] = session + 1
                df.loc[rowIndex,"Trial Number"] = trial + 1
                df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
                df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[1]
                df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[2]

                stimNodeDimensionSpeedMap = {
                    "Colour": None,
                    "Texture": None,
                    "Shape": None
                }
            
                if trial == 0: #for the very first trial generate 3 random numbers (integers)
                    color_int = random.randint(0,6)
                    texture_int = random.randint(0,6)
                    shape_int = random.randint(0,6)

                    df.loc[rowIndex, "Colour_stim"] = color_int
                    df.loc[rowIndex, "Texture_stim"] = texture_int
                    df.loc[rowIndex, "Shape_stim"] = shape_int

                    currentRow_colour = color_int
                    currentRow_texture = texture_int
                    currentRow_shape = shape_int

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]


                else:
                    color_int = float(f"{random.uniform(0,1):.2f}")
                    texture_int = float(f"{random.uniform(0,1):.2f}")
                    shape_int = float(f"{random.uniform(0,1):.2f}")

                    ##For the Colour dimension
                    
                    if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number is greater than stay probability, we can transition")

                        colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                        adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_colour == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                        
                        else:
                            if color_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            elif color_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_colour = transition_node
                            df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                        

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                        # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    ##For the texture dimension

                    if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number is greater than stay probability, we can transition")
                        texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                        adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_texture == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                        
                        else:
                            if texture_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {transition_node}","\n")

                            elif texture_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_texture = transition_node
                            df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                        # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                    ##For the shape dimension
                    
                    if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number is greater than stay probability, we can transition")
                        shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                        #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                        adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                        adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                        # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                        thresholds = [0.4, 0.7, 1]
                        transition_node  = None

                        
                        if currentRow_shape == 6:
                            transition_node = random.choice(adjacent_node_indices[0])
                            # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                        
                        else:
                            if shape_int <= 0.4:
                                transition_node = adjacent_node_indices[0][0]
                                # print(f"Node to which are transitioning: {shape_node}","\n")

                            elif shape_int <= 0.7:
                                transition_node = adjacent_node_indices[0][1]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                            else:
                                transition_node = adjacent_node_indices[0][2]
                                # print(f"Node to which are transitioning: {transition_node}","\n")
                        
                            currentRow_shape = transition_node
                            df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    else:
                        # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                        # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                    stimNodeDimensionSpeedMap["Colour"] = currentRow_colour
                    stimNodeDimensionSpeedMap["Texture"] = currentRow_texture
                    stimNodeDimensionSpeedMap["Shape"] = currentRow_shape

                    df.loc[rowIndex, "Slow_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Slow"]]
                    df.loc[rowIndex, "Medium_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Medium"]]
                    df.loc[rowIndex, "Fast_stim"] = stimNodeDimensionSpeedMap[dimensionSpeeds["Fast"]]
                
                rowIndex += 1 

            rowIndex_start_of_block = rowIndex - nTrials
            eligible_trials = []
            nodes_preced = None

            for trial in range(10, nTrials): 
                rowIndex_current = rowIndex_start_of_block + trial
                
                c_prev = df.loc[rowIndex_current-1, "Colour_stim"]
                t_prev = df.loc[rowIndex_current-1, "Texture_stim"]
                s_prev = df.loc[rowIndex_current-1, "Shape_stim"]
                nodes_preced = [c_prev,t_prev,s_prev]

                c_curr = df.loc[rowIndex_current, "Colour_stim"]
                t_curr = df.loc[rowIndex_current, "Texture_stim"]
                s_curr = df.loc[rowIndex_current, "Shape_stim"]

                full_transition = (
                    c_curr in possibleNeighbours[c_prev] and c_curr != c_prev and 
                    t_curr in possibleNeighbours[t_prev] and t_curr != t_prev and
                    s_curr in possibleNeighbours[s_prev] and s_curr != s_prev
                )

                distractor_colour = random.randint(0,6)
                distractor_texture = random.randint(0,6)
                distractor_shape = random.randint(0,6)

                while (distractor_colour == c_prev) or (distractor_texture == t_prev) or (distractor_shape == s_prev):
                    distractor_colour = random.randint(0,6)
                    distractor_texture = random.randint(0,6)
                    distractor_shape = random.randint(0,6)


                allMatch = (
                    distractor_colour in possibleNeighbours[c_prev] and distractor_colour!=c_prev,
                    distractor_texture in possibleNeighbours[t_prev] and distractor_texture!=t_prev,
                    distractor_shape in possibleNeighbours[s_prev] and distractor_shape!=s_prev
                )

                if sum(allMatch) < 3 and full_transition:  #append all the row indices where a trial is eligible
                    eligible_trials.append((trial, distractor_colour, distractor_texture, distractor_shape))

                # if 1 <= sum(allMatch) <= 2 and full_transition:
                #     eligible_trials.append((trial, distractor_colour, distractor_texture, distractor_shape))

            # print(len(eligible_trials))

            # num_choice_trials = 20
            # mingap = 5
            # maxgap = 8 
            # choice_trials = []

            # segment_size = nTrials // num_choice_trials
            # current_index = -mingap  

            # for i in range(num_choice_trials):
            #     start = max(i * segment_size, current_index + mingap)
            #     end = (i+1) * segment_size
            #     candidates = [t for t in eligible_trials if start <= t[0] < end and t[0] >= current_index + mingap]
            #     if candidates:
            #         within_maxgap = [t for t in candidates if t[0] <= current_index + maxgap]
            #         if within_maxgap:
            #             selected = random.choice(within_maxgap)
            #         else:
            #             # if none within maxgap, pick the closest candidate
            #             selected = min(candidates, key=lambda t: t[0])
            #         choice_trials.append(selected)
            #         current_index = selected[0]
            #     else:
            #         # fallback: pick the first eligible trial after current_index + mingap
            #         later_candidates = [t for t in eligible_trials if t[0] >= current_index + mingap]
            #         if later_candidates:
            #             selected = later_candidates[0]
            #             choice_trials.append(selected)
            #             current_index = selected[0]

            # print(choice_trials)
            
            # print(len(choice_trials))


            # num_choice_trials = 20
            # mingap = 5
            # maxgap = 10
            # last_trial = nTrials - 1
            # if last_trial not in [t[0]for t in eligible_trials]:
            #     last_trial = max([t[0] for t in eligible_trials if t[0]<nTrials])
            
            # eligible_trials_before_last = [t for t in eligible_trials if t[0]!=last_trial]

            # choice_trials = []
            # current = eligible_trials_before_last[0][0]
            # choice_trials.append(eligible_trials_before_last[0])

            # while len(choice_trials) < num_choice_trials:
            #     candidates = [t for t in eligible_trials_before_last if t[0] > current + mingap]
            #     if not candidates:
            #         remaining = [t for t in eligible_trials_before_last if t[0] > current]
            #         if remaining:
            #             current = remaining[0][0]
            #             choice_trials.append(remaining[0])
            #         break

            #     next_choice = candidates[0]
            #     for t in candidates:
            #         if t[0]<=current + maxgap:
            #             next_choice = t
            #             break
            #     choice_trials.append(next_choice)
            #     current = next_choice[0]

            # for t in eligible_trials:
            #     if t[0] == last_trial:
            #         choice_trials.append(t)
            #         break

            # print(len(choice_trials))

            K = 20           # number of choice trials required
            minGap = 7        # minimum gap between consecutive choice trials
            maxGap = 12       # maximum gap between consecutive choice trials

            # ---------- Clean eligible trials ----------
            # Remove any eligible entries where distractor equals the previous object's node
            eligible_clean = []
            for (trial, d_col, d_text, d_shape) in eligible_trials:
                prev_row = rowIndex_start_of_block + trial - 1
                prev_c = df.loc[prev_row, "Colour_stim"]
                prev_t = df.loc[prev_row, "Texture_stim"]
                prev_s = df.loc[prev_row, "Shape_stim"]
                # reject if distractor matches previous object's node on any dimension
                if (d_col == prev_c) or (d_text == prev_t) or (d_shape == prev_s):
                    continue
                # also ensure distractor is not fully consistent (just in case): allow 0-2 matches
                allMatch = (
                    (d_col in possibleNeighbours[prev_c]) and (d_col != prev_c),
                    (d_text in possibleNeighbours[prev_t]) and (d_text != prev_t),
                    (d_shape in possibleNeighbours[prev_s]) and (d_shape != prev_s)
                )
                if sum(allMatch) <= 2:   # maintain original rule: not all three consistent
                    eligible_clean.append((trial, d_col, d_text, d_shape))

            # If there are fewer than K eligibles, we will need to relax later (but try DP first)
            positions = [t[0] for t in eligible_clean]
            m = len(positions)

            if m < K:
                print(f"[warning] only {m} eligible_clean trials available (need {K}). Will try relax/fill later.")

            # ---------- DP solver (find K positions satisfying minGap..maxGap) ----------
            def find_sequence_dp(positions, K, minG, maxG):
                m = len(positions)
                if m < K:
                    return None
                # dp[l][i] = predecessor index (in positions list) for length l sequence ending at positions[i]
                dp = [[None]*m for _ in range(K+1)]
                # base: length 1 sequences exist for any position
                for i in range(m):
                    dp[1][i] = -1   # -1 marks the start (no predecessor)
                # build up
                for l in range(2, K+1):
                    for i in range(m):
                        dp[l][i] = None
                        # find a p < i with dp[l-1][p] != None and gap constraint satisfied
                        for p in range(i):
                            if dp[l-1][p] is not None:
                                gap = positions[i] - positions[p]
                                if minG <= gap <= maxG:
                                    dp[l][i] = p
                                    break
                # find any i such that dp[K][i] is not None
                for i in range(m):
                    if dp[K][i] is not None:
                        # reconstruct sequence of indices into 'positions'
                        seq_idx = []
                        cur = i
                        for l in range(K, 0, -1):
                            seq_idx.append(cur)
                            cur = dp[l][cur]
                            if cur == -1:
                                break
                        seq_idx.reverse()
                        # return list of indices (into eligible_clean)
                        return seq_idx
                return None

            # ---------- Try strict DP, then relax if needed ----------
            chosen_idx = None
            # Try gradually increasing maxGap first (prefer to keep minGap)
            for relax_max in range(maxGap, maxGap + 21):   # allow expansion up to +20 if needed
                chosen_idx = find_sequence_dp(positions, K, minGap, relax_max)
                if chosen_idx is not None:
                    used_minGap, used_maxGap = minGap, relax_max
                    break

            # If that failed, try reducing minGap as last resort (while resetting maxGap attempts)
            if chosen_idx is None:
                for relax_min in range(minGap-1, 0, -1):   # lower minGap down toward 1
                    for relax_max in range(maxGap, maxGap + 21):
                        chosen_idx = find_sequence_dp(positions, K, relax_min, relax_max)
                        if chosen_idx is not None:
                            used_minGap, used_maxGap = relax_min, relax_max
                            break
                    if chosen_idx is not None:
                        break

            # ---------- Fallback greedy fill (if DP still failed) ----------
            if chosen_idx is None:
                print("[warning] DP failed to find a sequence under allowed relaxations. Using greedy fill fallback.")
                chosen = []
                used_positions = set()
                current_pos = -1000
                # greedy pick K items respecting minGap if possible
                for (trial, d_c, d_t, d_s) in eligible_clean:
                    if len(chosen) >= K:
                        break
                    if trial >= current_pos + minGap:
                        chosen.append((trial, d_c, d_t, d_s))
                        current_pos = trial
                # If still not enough, take remaining eligibles closest to end while keeping order
                if len(chosen) < K:
                    for (trial, d_c, d_t, d_s) in eligible_clean:
                        if len(chosen) >= K:
                            break
                        if (trial, d_c, d_t, d_s) not in chosen:
                            chosen.append((trial, d_c, d_t, d_s))
                # ensure we have K entries
                chosen = chosen[:K]
                # convert chosen to indices into eligible_clean
                chosen_idx = [eligible_clean.index(item) for item in chosen]

            # ---------- Build the final list of selected choice trials ----------
            selected_choice_trials = [eligible_clean[i] for i in chosen_idx]  # list of (trial, dcol, dtex, dshape)
            selected_choice_positions = [t[0] for t in selected_choice_trials]

            # Sort them in ascending trial order to be safe
            selected_choice = sorted(selected_choice_trials, key=lambda x: x[0])

            # Report if any relaxation happened
            try:
                if used_minGap != minGap or used_maxGap != maxGap:
                    print(f"[info] used relaxed constraints: minGap={used_minGap}, maxGap={used_maxGap}")
            except NameError:
                # no relax attempted/needed
                pass

            print(f"[info] Selected {len(selected_choice)} choice trials at indices: {[t[0] for t in selected_choice]}")

            # ---------- Assign distractors and update dataframe for each chosen trial ----------

            for index, (trial, d_col, d_text, d_shape) in enumerate(selected_choice, start=1):
                rowIndex_current = rowIndex_start_of_block + trial

                df.loc[rowIndex_current, "Colour_distractor"] = d_col
                df.loc[rowIndex_current, "Texture_distractor"] = d_text
                df.loc[rowIndex_current, "Shape_distractor"] = d_shape
                df.loc[rowIndex_current, "Choice Trial Index"] = index


                c_stim = df.loc[rowIndex_current, "Colour_stim"]
                t_stim = df.loc[rowIndex_current, "Texture_stim"]
                s_stim = df.loc[rowIndex_current, "Shape_stim"]


                stimNodeDimensionSpeedMap = {
                    "Colour": c_stim,
                    "Texture": t_stim,
                    "Shape": s_stim
                }
                distractorNodeDimensionsMap = {
                    "Colour": d_col,
                    "Texture": d_text,
                    "Shape": d_shape
                }
                df.loc[rowIndex_current, "Slow_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Slow"]]
                df.loc[rowIndex_current, "Medium_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Medium"]]
                df.loc[rowIndex_current, "Fast_distractor"] = distractorNodeDimensionsMap[dimensionSpeeds["Fast"]]

               


                for col, dim in zip(["Distractor_shares_colour_target", "Distractor_shares_texture_target","Distractor_shares_shape_target"], 
                    ["Colour", "Texture", "Shape"]):
                    df.loc[rowIndex_current, col] = 1 if distractorNodeDimensionsMap[dim] == stimNodeDimensionSpeedMap[dim] else 0
                for col,dim in zip(["Distractor_shares_slow_target", "Distractor_shares_medium_target", "Distractor_shares_fast_target"], 
                                   ["Slow", "Medium", "Fast"]):
                    df.loc[rowIndex_current, col] = 1 if distractorNodeDimensionsMap[dimensionSpeeds[dim]] == stimNodeDimensionSpeedMap[dimensionSpeeds[dim]] else 0




                #assign 1 if distractor is consistent on any dimension compared to the previous object, else 0
                prevRow = rowIndex_current - 1
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_consistent_colour", "Distractor_consistent_texture", "Distractor_consistent_shape"],
                    ["Colour_stim", "Texture_stim", "Shape_stim"],
                    ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                ) :
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dist_dim] in possibleNeighbours[df.loc[prevRow, stim_dim]] and df.loc[rowIndex_current, dist_dim] != df.loc[prevRow, stim_dim] else 0
                #assign 1 distractor is consistent on any dimension with regards to speed, compared to the previous object, else 0
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_consistent_slow", "Distractor_consistent_medium", "Distractor_consistent_fast"],
                    ["Slow_stim", "Medium_stim", "Fast_stim"],
                    ["Slow_distractor", "Medium_distractor", "Fast_distractor"]
                ):
                    df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current, dist_dim] in possibleNeighbours[df.loc[prevRow, stim_dim]] and df.loc[rowIndex_current, dist_dim] != df.loc[prevRow, stim_dim] else 0
                            
                # #assign 1 if distractor shares the exact node/feature compared to the target, else 0 
                # for col, stim_dim, dist_dim in zip(
                #     ["Distractor_shares_colour_target", "Distractor_shares_texture_target", "Distractor_shares_shape_target"],
                #     ["Colour_stim", "Texture_stim", "Shape_stim"],
                #     ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                # ) :
                #     df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current,dist_dim] == df.loc[rowIndex_current, stim_dim] else 0

                # #assign 1 if distractor shares the exact node/feature regarding speed compared to the target, else 0
                # for col, stim_dim, dist_dim in zip(
                #     ["Distractor_shares_slow_target", "Distractor_shares_medium_target", "Distractor_shares_fast_target"],
                #     ["Slow_stim", "Medium_stim", "Shape_stim"],
                #     ["Slow_distractor", "Medium_distractor", "Fast_distractor"]
                # ) :
                #     df.loc[rowIndex_current, col] = 1 if df.loc[rowIndex_current,dist_dim] == df.loc[rowIndex_current, stim_dim] else 0

                #assign 1 if distractor shares the exact node/feature compared to the previous object, else 0
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_shares_colour_object", "Distractor_shares_texture_object", "Distractor_shares_shape_object"],
                    ["Colour_stim", "Texture_stim", "Shape_stim"],
                    ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                ):
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dist_dim] == df.loc[prevRow,stim_dim] else 0

                #assign 1 if distractor shares the exact node/feature with regards to speed compared to the previous object, else 0
                for col, stim_dim, dist_dim in zip(
                    ["Distractor_shares_slow_object", "Distractor_shares_medium_object", "Distractor_shares_fast_object"],
                    ["Colour_stim", "Texture_stim", "Shape_stim"],
                    ["Colour_distractor", "Texture_distractor", "Shape_distractor"]
                ):
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dist_dim] == df.loc[prevRow,stim_dim] else 0

                #assign 1 and 0 if target shares exact features of the previous object
                for col, dim in zip(["Target_shares_colour","Target_shares_texture","Target_shares_shape"], 
                                                  ["Colour_stim", "Texture_stim","Shape_stim"]):
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dim] == df.loc[prevRow, dim] else 0

                #assign 1 and 0 if target shares exact features of the previous object
                for col, dim in zip(["Target_shares_slow","Target_shares_medium","Target_shares_fast"], 
                                                  ["Slow_stim", "Medium_stim","Fast_stim"]):
                    df.loc[rowIndex_current,col] = 1 if df.loc[rowIndex_current, dim] == df.loc[prevRow, dim] else 0
                 
                    
        df.iloc[:,0] = participant + 1
        df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        #df.to_csv(f"DesignFile_{participant+1}_session{session+1}.csv", index=False)
                




[info] used relaxed constraints: minGap=6, maxGap=12
[info] Selected 20 choice trials at indices: [13, 19, 26, 32, 38, 47, 56, 66, 72, 78, 84, 92, 102, 114, 120, 129, 140, 151, 158, 166]
[info] used relaxed constraints: minGap=6, maxGap=24
[info] Selected 20 choice trials at indices: [13, 23, 29, 35, 43, 49, 59, 66, 73, 86, 93, 100, 108, 114, 121, 127, 136, 147, 171, 184]
[info] used relaxed constraints: minGap=6, maxGap=13
[info] Selected 20 choice trials at indices: [11, 17, 25, 32, 39, 45, 53, 59, 70, 76, 86, 98, 104, 114, 126, 139, 150, 163, 171, 184]
[info] used relaxed constraints: minGap=6, maxGap=13
[info] Selected 20 choice trials at indices: [14, 20, 31, 38, 48, 54, 62, 75, 87, 93, 99, 105, 113, 122, 132, 141, 151, 164, 172, 179]
[info] used relaxed constraints: minGap=5, maxGap=18
[info] Selected 20 choice trials at indices: [11, 17, 23, 33, 41, 46, 51, 56, 62, 69, 78, 83, 88, 93, 103, 118, 124, 130, 148, 158]
[info] used relaxed constraints: minGap=5, maxGap=16
[info] Selec

<h3>Sanity Check Code</h3>

In [ ]:
df["Distractor_consistent_colour"].unique()

array([nan,  0.,  1.])

In [ ]:
def checkConsistency(df):
    dist_all_consistent = (
        (df["Distractor_consistent_colour"] == 1) &
        (df["Distractor_consistent_texture"] == 1) &
        (df["Distractor_consistent_shape"] == 1)
    ).any()

    dist_shares_feature = (
        (df["Distractor_shares_colour_object"] == 1) |
        (df["Distractor_shares_texture_object"] == 1) |
        (df["Distractor_shares_shape_object"] == 1)
    ).any()

    if dist_all_consistent:
        print("There are distractors which have all dimensions consistent")
    if dist_shares_feature:
            print("There are distractors which share a feature with previous object")

    if not dist_all_consistent and not dist_shares_feature:
        print("Distractors are clean")

In [ ]:
designFileList = os.listdir("../Experiment (No Backend)/public/DesignFiles")
designFileList.remove("DesignFileList.json")

for index, file in enumerate(designFileList):
    df = pd.read_csv(f"../Experiment (No Backend)/public/DesignFiles/{file}")
    checkConsistency(df)

Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractors are clean
Distractor

<h1>Version 2 of Experiment</h1>


In [2]:
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(6)
edges = [
    (0,1),
    (0,5),
    (1,0),
    (1,2),
    (2,1),
    (2,3),
    (3,2),
    (3,4),
    (4,3),
    (4,5),
    (5,4),
    (5,0)

]

G = Graph(nodes,edges)

<img src="../GraphStructure_V2.png"/>

In [3]:
def generate_transitionMatrix(graph,stay_probability=0):
    """Function to create the transition matrix. We assume that the graph has already been created and the 
    relevant nodes and edges are clearly defined. """

    #get the number of nodes
    numberofNodes = len(graph.nodes)
    #get shape of the transition matrix 
    matrix_shape = (numberofNodes, numberofNodes)
    #create the adjacent matrix with zeros
    adj_matrix = np.zeros(matrix_shape)

    for edge in graph.edges:
        node1,node2 = edge[0],edge[1]
        adj_matrix[node1][node2] += 1
        adj_matrix[node2][node1] += 1
    adj_matrix = adj_matrix/2 #just to make everything 1s and 0s
    
    #lets keep the calculation of the transition probabilites dynamic based off the number of adjacent nodes from
    #a starting node and the stay probability of that starting node
    #therefore what we should have is: transition probability = (1 - stay probability) / number of adjacent nodes

    np.fill_diagonal(adj_matrix,stay_probability) #set the stay probability in the adj matrix for the for loop 
    transition_matrix = np.copy(adj_matrix)

    #lets get the number of neighbours for each node by reading off the total number of 1s (adjacent nodes)
    for index,row in enumerate(adj_matrix):
        #we assume that the loops of a node to itself are already defined in the edges of the graph
        number_adjacent_nodes = np.count_nonzero(row == 1)
        transition_probability = (1 - stay_probability) / (number_adjacent_nodes)

        #adjust the decimal places
        transition_probability

        #replace the 1s (adjacent nodes) with their transition probability
        transition_matrix[index] = np.where(row ==1, f"{transition_probability:.2f}", row) #formatting the transition probability to two decimal places

    #set the stay probability of the matrix for each node 
    np.fill_diagonal(transition_matrix,stay_probability)
    #replace the diagonals with zeros for the adjacent matrix
    np.fill_diagonal(adj_matrix, 0)

    return adj_matrix, transition_matrix
        



In [4]:
colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=0.1)
colour_adjmatrix, colour_transition_matrix

(array([[0., 1., 0., 0., 0., 1.],
        [1., 0., 1., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 1., 0.]]),
 array([[0.1 , 0.45, 0.  , 0.  , 0.  , 0.45],
        [0.45, 0.1 , 0.45, 0.  , 0.  , 0.  ],
        [0.  , 0.45, 0.1 , 0.45, 0.  , 0.  ],
        [0.  , 0.  , 0.45, 0.1 , 0.45, 0.  ],
        [0.  , 0.  , 0.  , 0.45, 0.1 , 0.45],
        [0.45, 0.  , 0.  , 0.  , 0.45, 0.1 ]]))

In [7]:
import random
import numpy as np
import pandas as pd

# --- keep your helper functions available ---
# generate_transitionMatrix(G, stay_probability=...)  # assumed available
# (If you have other helpers, keep them available)

# ----------------------------------------------------------------
# Parameters & graph neighbours (from your latest version)
# ----------------------------------------------------------------
master_df = []

nTrials = 200
nParticipants = 50
nBlocks = 8
nSessions = 2
stayProbs = [0.1, 0.3, 0.5]

possibleNeighbours = {
    0: [0, 1, 5],
    1: [1, 0, 2],
    2: [2, 1, 3],
    3: [3, 2, 4],
    4: [4, 3, 5],
    5: [5, 4, 0],
}

colours = ["Blue", "Red", "Yellow", "Pink", "Green", "Grey"]
textures = ["Bricks", "Dots", "Grid", "Waves", "Bottles", "VerticalLines"]
shapes = ["Rectangle", "Semicircle", "Circle", "Hexagon", "Star", "Triangle"]

# columns you want in the final df (kept similar to yours)
columns = [
    "Participant_ID", "Block Number", "Session Number", "Trial Number",
    "Stay_Probability(Colour)", "Stay_Probability(Texture)", "Stay_Probability(Shape)",
    "Colour_Graph", "Texture_Graph", "Shape_Graph", "Colour_code", "Texture_code", "Shape_code",
    "Slow_stim", "Medium_stim", "Fast_stim", "Colour_stim", "Texture_stim", "Shape_stim",
    "Choice Trial Index", "Distractor_colour_code", "Distractor_texture_code", "Distractor_shape_code", "Colour_distractor", "Texture_distractor", "Shape_distractor",
    "Slow_distractor", "Medium_distractor", "Fast_distractor",
    "Target_shares_colour", "Target_shares_texture", "Target_shares_shape",
    "Target_shares_slow", "Target_shares_medium", "Target_shares_fast",
    "Distractor_consistent_colour", "Distractor_consistent_texture", "Distractor_consistent_shape",
    "Distractor_consistent_slow", "Distractor_consistent_medium", "Distractor_consistent_fast",
    "Distractor_shares_colour_target", "Distractor_shares_texture_target", "Distractor_shares_shape_target",
    "Distractor_shares_slow_target", "Distractor_shares_medium_target", "Distractor_shares_fast_target",
    "Distractor_shares_colour_object", "Distractor_shares_texture_object", "Distractor_shares_shape_object",
    "Distractor_shares_slow_object", "Distractor_shares_medium_object", "Distractor_shares_fast_object"
]

# ----------------------------------------------------------------
# Main loop: build records then wrap into a DataFrame.
# ----------------------------------------------------------------
for participant in range(nParticipants):
    # shuffle graphs for THIS participant
    random.shuffle(colours)
    random.shuffle(textures)
    random.shuffle(shapes)

    # shuffle stay probabilities and build speed mapping
    random.shuffle(stayProbs)
    slow = np.argmax(stayProbs)
    medium = np.where(np.array(stayProbs) == 0.3)[0][0]
    fast = np.argmin(stayProbs)
    dimensionSpeedMap = {0: "Colour", 1: "Texture", 2: "Shape"}
    dimensionSpeeds = {
        "Slow": dimensionSpeedMap[slow],
        "Medium": dimensionSpeedMap[medium],
        "Fast": dimensionSpeedMap[fast]
    }

    # build transition matrices (assumed function exists)
    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G, stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G, stay_probability=stayProbs[2])

    

    # ---- STIMULUS GENERATION PASS (fast, list-of-dicts) ----
    for session in range(nSessions):
        # Records list (fast append)
        records = []
        for block in range(nBlocks):
            # start of block: current nodes are None (will be set for trial 0)
            current_colour = None
            current_texture = None
            current_shape = None

            for trial in range(nTrials):
                # initial record with graph lists included (store lists as objects)
                rec = {
                    "Participant_ID": None,  # set later in df
                    "Block Number": block + 1,
                    "Session Number": session + 1,
                    "Trial Number": trial + 1,
                    "Stay_Probability(Colour)": stayProbs[0],
                    "Stay_Probability(Texture)": stayProbs[1],
                    "Stay_Probability(Shape)": stayProbs[2],
                    "Colour_Graph": list(colours),   # store the shuffled graph mapping
                    "Texture_Graph": list(textures),
                    "Shape_Graph": list(shapes),
                    "Colour_code": None,
                    "Texture_code": None,
                    "Shape_code": None,
                    # filler: will fill stim/distractor fields below
                    "Slow_stim": None, "Medium_stim": None, "Fast_stim": None,
                    "Colour_stim": None, "Texture_stim": None, "Shape_stim": None,
                    "Choice Trial Index": None,
                    "Distractor_colour_code":None, "Distractor_texture_code":None, "Distractor_shape_code":None,
                    "Colour_distractor": None, "Texture_distractor": None, "Shape_distractor": None,
                    "Slow_distractor": None, "Medium_distractor": None, "Fast_distractor": None,
                    # shares/consistent placeholders
                    "Target_shares_colour": 0, "Target_shares_texture": 0, "Target_shares_shape": 0,
                    "Target_shares_slow": 0, "Target_shares_medium": 0, "Target_shares_fast": 0,
                    "Distractor_consistent_colour": 0, "Distractor_consistent_texture": 0, "Distractor_consistent_shape": 0,
                    "Distractor_consistent_slow": 0, "Distractor_consistent_medium": 0, "Distractor_consistent_fast": 0,
                    "Distractor_shares_colour_target": 0, "Distractor_shares_texture_target": 0, "Distractor_shares_shape_target": 0,
                    "Distractor_shares_slow_target": 0, "Distractor_shares_medium_target": 0, "Distractor_shares_fast_target": 0,
                    "Distractor_shares_colour_object": 0, "Distractor_shares_texture_object": 0, "Distractor_shares_shape_object": 0,
                    "Distractor_shares_slow_object": 0, "Distractor_shares_medium_object": 0, "Distractor_shares_fast_object": 0
                }

                # ---- stimulus generation logic (keeps your transition logic) ----
                if trial == 0:
                    # initialize random node indices 0..5
                    current_colour = random.randint(0, 5)
                    current_texture = random.randint(0, 5)
                    current_shape = random.randint(0, 5)
                else:
                    # sample random numbers and decide transitions using transition matrices
                    rcol = random.random()
                    rtex = random.random()
                    rsha = random.random()

                    # Colour
                    if rcol > stayProbs[0]:
                        row = colour_transition_matrix[current_colour]
                        # eligible neighbor indices: exclude self index (stay)
                        adj_idx = np.where(np.arange(len(row)) != current_colour)[0]
                        # filter only those with non-zero probability
                        adj_idx = adj_idx[row[adj_idx] > 0]
                        if len(adj_idx):
                            probs = row[adj_idx].astype(float)
                            probs = probs / probs.sum()
                            current_colour = int(np.random.choice(adj_idx, p=probs))
                        # else keep same
                    # else keep same

                    # Texture
                    if rtex > stayProbs[1]:
                        row = texture_transition_matrix[current_texture]
                        adj_idx = np.where(np.arange(len(row)) != current_texture)[0]
                        adj_idx = adj_idx[row[adj_idx] > 0]
                        if len(adj_idx):
                            probs = row[adj_idx].astype(float)
                            probs = probs / probs.sum()
                            current_texture = int(np.random.choice(adj_idx, p=probs))

                    # Shape
                    if rsha > stayProbs[2]:
                        row = shape_transition_matrix[current_shape]
                        adj_idx = np.where(np.arange(len(row)) != current_shape)[0]
                        adj_idx = adj_idx[row[adj_idx] > 0]
                        if len(adj_idx):
                            probs = row[adj_idx].astype(float)
                            probs = probs / probs.sum()
                            current_shape = int(np.random.choice(adj_idx, p=probs))

                # write stimulus nodes into the record
                rec["Colour_stim"] = int(current_colour)
                rec["Texture_stim"] = int(current_texture)
                rec["Shape_stim"] = int(current_shape)
                rec["Colour_code"] = colours[int(current_colour)]
                rec["Texture_code"] = textures[int(current_texture)]
                rec["Shape_code"] = shapes[int(current_shape)]


                # speed-mapped stim values
                stim_by_dim = {"Colour": current_colour, "Texture": current_texture, "Shape": current_shape}
                rec["Slow_stim"] = stim_by_dim[dimensionSpeeds["Slow"]]
                rec["Medium_stim"] = stim_by_dim[dimensionSpeeds["Medium"]]
                rec["Fast_stim"] = stim_by_dim[dimensionSpeeds["Fast"]]


                # append record
                records.append(rec)

    # End of stimulus-generation pass for participant/session/block
    # Build DataFrame once (fast)
        df = pd.DataFrame(records, columns=columns)

        # Set participant id
        df["Participant_ID"] = participant + 1

       # ---- CHOICE TRIAL SELECTION & DISTRACTOR ASSIGNMENT (SIMULTANEOUS CHECK) ----
        nChoiceTrials = 18
        min_gap, max_gap = 8, 10
        start_after = random.randint(9, 10)  # earliest trial (9 or 10)

        for block_num in df["Block Number"].unique():
            # keep a reset_index view so we can get original df indices via "index"
            block_df = df[df["Block Number"] == block_num].reset_index()
            nTrials = len(block_df)
            if nTrials <= start_after:
                continue

            # 1) Build list of eligible trials: full transitions AND have at least one valid distractor
            eligible_trials = []  # will store (trial_idx_in_block, (dcol, dtex, dshape))
            for i in range(1, nTrials):  # skip first trial (no previous)
                prev = block_df.loc[i - 1]
                curr = block_df.loc[i]

                # full transition check (target does NOT share any feature with previous)
                if (
                    curr["Colour_stim"] == prev["Colour_stim"]
                    or curr["Texture_stim"] == prev["Texture_stim"]
                    or curr["Shape_stim"] == prev["Shape_stim"]
                ):
                    continue

                # try to find a valid distractor (not all 3 consistent with previous)
                c_prev = int(prev["Colour_stim"])
                t_prev = int(prev["Texture_stim"])
                s_prev = int(prev["Shape_stim"])

                valid_distractor = None
                for _ in range(200):  # attempts
                    distractor_colour = random.randint(0, 5)
                    distractor_texture = random.randint(0, 5)
                    distractor_shape = random.randint(0, 5)

                    consistent_colour = int(distractor_colour in possibleNeighbours[c_prev])
                    consistent_texture = int(distractor_texture in possibleNeighbours[t_prev])
                    consistent_shape = int(distractor_shape in possibleNeighbours[s_prev])
                    total_consistent = consistent_colour + consistent_texture + consistent_shape

                    if total_consistent < 3:
                        valid_distractor = (distractor_colour, distractor_texture, distractor_shape)
                        break

                # if found, add to eligible list
                if valid_distractor is not None:
                    eligible_trials.append((i, valid_distractor))

            # restrict to after start_after
            eligible_trials = [(i, d) for (i, d) in eligible_trials if i >= start_after]
            if not eligible_trials:
                # nothing to do for this block
                continue

            # make a dict for quick lookup later
            eligible_dict = {i: d for (i, d) in eligible_trials}

            # 2) Strictly sample trials with gaps between min_gap..max_gap
            chosen = []
            last = start_after - min_gap  # so first low = start_after
            attempts_guard = 0
            while len(chosen) < nChoiceTrials and attempts_guard < 1000:
                attempts_guard += 1
                low = last + min_gap
                high = last + max_gap
                # collect eligible indices in [low, high]
                window_candidates = [i for (i, _) in eligible_trials if low <= i <= high and i not in chosen]
                if not window_candidates:
                    # if no candidate in window, try to find the next eligible >= low (but still < nTrials)
                    next_candidates = [i for (i, _) in eligible_trials if i >= low and i not in chosen]
                    if not next_candidates:
                        # cannot extend further while preserving spacing
                        break
                    next_choice = next_candidates[0]  # earliest eligible after low
                else:
                    # pick randomly among window candidates to avoid deterministic bias
                    next_choice = random.choice(window_candidates)

                chosen.append(next_choice)
                last = next_choice

            # If we couldn't reach nChoiceTrials while preserving strict spacing, we keep what we have (strict)
            chosen = sorted(chosen)

            # 3) Assign distractors and all derived flags — use original df indices
            choice_idx = 1
            for trial_idx in chosen:
                abs_index = block_df.loc[trial_idx, "index"]        # index in original df
                prev_index = block_df.loc[trial_idx - 1, "index"]  # previous trial index in original df

                # retrieve the pre-generated distractor tuple for this trial
                distractor_colour, distractor_texture, distractor_shape = eligible_dict[trial_idx]

                # write distractor identifiers & codes
                df.loc[abs_index, "Choice Trial Index"] = choice_idx
                df.loc[abs_index, "Colour_distractor"] = distractor_colour
                df.loc[abs_index, "Texture_distractor"] = distractor_texture
                df.loc[abs_index, "Shape_distractor"] = distractor_shape
                df.loc[abs_index, "Distractor_colour_code"] = colours[distractor_colour]
                df.loc[abs_index, "Distractor_texture_code"] = textures[distractor_texture]
                df.loc[abs_index, "Distractor_shape_code"] = shapes[distractor_shape]

                # speed-mapped distractor values
                stim_map = {"Colour": distractor_colour, "Texture": distractor_texture, "Shape": distractor_shape}
                df.loc[abs_index, "Slow_distractor"] = stim_map[dimensionSpeeds["Slow"]]
                df.loc[abs_index, "Medium_distractor"] = stim_map[dimensionSpeeds["Medium"]]
                df.loc[abs_index, "Fast_distractor"] = stim_map[dimensionSpeeds["Fast"]]

                # consistency flags (with previous stimulus's nodes)
                c_prev = int(df.loc[prev_index, "Colour_stim"])
                t_prev = int(df.loc[prev_index, "Texture_stim"])
                s_prev = int(df.loc[prev_index, "Shape_stim"])

                df.loc[abs_index, "Distractor_consistent_colour"] = int(distractor_colour in possibleNeighbours[c_prev])
                df.loc[abs_index, "Distractor_consistent_texture"] = int(distractor_texture in possibleNeighbours[t_prev])
                df.loc[abs_index, "Distractor_consistent_shape"] = int(distractor_shape in possibleNeighbours[s_prev])

                # derived consistency by speed (Colour/Texture/Shape remapped by dimensionSpeeds)
                df.loc[abs_index, "Distractor_consistent_slow"] = df.loc[abs_index, f"Distractor_consistent_{dimensionSpeeds['Slow'].lower()}"]
                df.loc[abs_index, "Distractor_consistent_medium"] = df.loc[abs_index, f"Distractor_consistent_{dimensionSpeeds['Medium'].lower()}"]
                df.loc[abs_index, "Distractor_consistent_fast"] = df.loc[abs_index, f"Distractor_consistent_{dimensionSpeeds['Fast'].lower()}"]

                # distractor shares target flags (identical)
                df.loc[abs_index, "Distractor_shares_colour_target"] = int(df.loc[abs_index, "Colour_stim"] == distractor_colour)
                df.loc[abs_index, "Distractor_shares_texture_target"] = int(df.loc[abs_index, "Texture_stim"] == distractor_texture)
                df.loc[abs_index, "Distractor_shares_shape_target"] = int(df.loc[abs_index, "Shape_stim"] == distractor_shape)

                df.loc[abs_index, "Distractor_shares_slow_target"] = df.loc[abs_index, f"Distractor_shares_{dimensionSpeeds['Slow'].lower()}_target"]
                df.loc[abs_index, "Distractor_shares_medium_target"] = df.loc[abs_index, f"Distractor_shares_{dimensionSpeeds['Medium'].lower()}_target"]
                df.loc[abs_index, "Distractor_shares_fast_target"] = df.loc[abs_index, f"Distractor_shares_{dimensionSpeeds['Fast'].lower()}_target"]

                # ---- YOUR MISSING ASSIGNMENTS: target shares with previous object ----
                # direct-dimension shares (Colour/Texture/Shape)
                for col, dim in zip(
                    ["Target_shares_colour", "Target_shares_texture", "Target_shares_shape"],
                    ["Colour_stim", "Texture_stim", "Shape_stim"],
                ):
                    df.loc[abs_index, col] = 1 if df.loc[abs_index, dim] == df.loc[prev_index, dim] else 0

                # slow/medium/fast shares
                for col, dim in zip(
                    ["Target_shares_slow", "Target_shares_medium", "Target_shares_fast"],
                    ["Slow_stim", "Medium_stim", "Fast_stim"],
                ):
                    df.loc[abs_index, col] = 1 if df.loc[abs_index, dim] == df.loc[prev_index, dim] else 0

                choice_idx += 1




    # Done: you can now save df for this participant/session or append to master
    # Example: save per session or per participant as you want
        #df.to_csv(f"DesignFile_participant{participant+1}_session{session+1}.csv", index=False)
        #df.to_csv(f"../Experiment (No Backend)/public/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)
        df.to_csv(f"C:/jatos_win_java/study_assets_root/7d328c55-7802-4e40-906d-2d2e3905cd0e/DesignFiles/DesignFile_{participant+1}_session{session+1}.csv", index=False)



In [ ]:
def checkConsistency(df):
    dist_all_consistent = (
        (df["Distractor_consistent_colour"] == 1) &
        (df["Distractor_consistent_texture"] == 1) &
        (df["Distractor_consistent_shape"] == 1)
    ).any()

    dist_shares_feature = (
        (df["Distractor_shares_colour_object"] == 1) |
        (df["Distractor_shares_texture_object"] == 1) |
        (df["Distractor_shares_shape_object"] == 1)
    ).any()         

    if dist_all_consistent:
        print("There are distractors which have all dimensions consistent")
    if dist_shares_feature:
            print("There are distractors which share a feature with previous object")

    if not dist_all_consistent and not dist_shares_feature:
        print("Distractors are clean")

    choice_trials = df[df["Choice Trial Index"].notna()]
    shares_any = []
    for idx in choice_trials.index:
        if idx == 0:  # skip the very first trial
            continue
        prev_idx = idx - 1

        same_colour = df.loc[idx, "Colour_stim"] == df.loc[prev_idx, "Colour_stim"]
        same_texture = df.loc[idx, "Texture_stim"] == df.loc[prev_idx, "Texture_stim"]
        same_shape = df.loc[idx, "Shape_stim"] == df.loc[prev_idx, "Shape_stim"]

        if same_colour or same_texture or same_shape:
            shares_any.append(idx)

    if shares_any:
        print(f"⚠️ {len(shares_any)} choice trials have targets that share features with the previous object.")
        print("  Indices:", shares_any)
    else:
        print("✅ All choice trial targets are unique from their preceding stimulus across all dimensions.")



In [19]:
dir = "C:/jatos_win_java/study_assets_root/7d328c55-7802-4e40-906d-2d2e3905cd0e/DesignFiles"
files = os.listdir(dir)

for file in files:
    if file.endswith(".json"):
        continue
    else:
        df = pd.read_csv(f"{dir}/{file}")
        checkConsistency(df)

Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across all dimensions.
Distractors are clean
✅ All choice trial targets are unique from their preceding stimulus across

In [6]:
import pandas as pd
df = pd.read_csv("data1.csv")

checkConsistency(df)

Distractors are clean
⚠️ 5 choice trials have targets that share features with the previous object.
  Indices: [1727, 1746, 2044, 2352, 3318]


<h3>Creating a json file with the list of the folders in the design file folder</h3>

In [ ]:
import os
import json

In [ ]:
designFileList = os.listdir("../Experiment (No Backend)/public/DesignFiles")
designFileList.remove("DesignFileList.json")

with open("../Experiment (No Backend)/public/DesignFiles/DesignFileList.json", "w") as jsonFile:
    json.dump(designFileList, jsonFile)

